In [1]:
import json
import logging
import os
import sys
import urllib
import warnings
from datetime import datetime, date

import pandas as pd
from sqlalchemy import create_engine, text
import json 


### Prepare the data for Band Analysis (RedZoneV2-based vs. Anson's IBV Score-based)

#### Imoport the data with decoded model output JSON

In [ ]:
# import the data and try to parse the JSON output
server = '192.168.1.15'
username = 'Junchen'
password = '9DFXjhhlR3k5'
database = 'BankuityPostOnboarding'

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server},1433;DATABASE={database};UID={username};PWD={password};"
    "Encrypt=yes;TrustServerCertificate=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

cnxn = engine.connect()

query1 = f""" 
SELECT *,
          CONVERT(VARCHAR(MAX),
                  DECOMPRESS(ResponseJSON)
         ) AS ResponseJSONText
FROM BankuityPostOnboarding.dbo.SpeedyAnalysis
WHERE ExperimentName = 'loonie_rerun_V3';
"""

df_rerun = pd.read_sql(query1, cnxn)

In [4]:
print(df_rerun.shape)

(11474, 11)


#### Write the ResponseJSONText to Local (CAREFULL TO RUN!!!!!!!)


In [ ]:
""" output_dir = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_output_JSONs_V3"
os.makedirs(output_dir, exist_ok=True)

for token, response_text in zip(df_rerun.IBVToken.values, df_rerun.ResponseJSONText.values):
    try:
        data = json.loads(response_text) if isinstance(response_text, str) else response_text
        filename = os.path.join(output_dir, f"ModelResponse_{token}.json")
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, separators=(',', ': '), ensure_ascii=False)
    except Exception as e:
        print(token, e) """

#### Extract the RedZoneV2 RiskScore from the JSONs

In [5]:
import re

input_dir = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_output_JSONs_V3"
rows = []

for filename in os.listdir(input_dir):
    if not filename.endswith(".json"):
        continue
    match = re.search(r"_(\d+)\.json$", filename)
    if not match:
        continue
    ibv_status_id = int(match.group(1))
    file_path = os.path.join(input_dir, filename)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            payload = json.load(f)
        customer = (payload or {}).get("customerInfo") or {}
        scores = customer.get("scores", {})
        redzone = scores.get("redZoneV2", {})
        model_score = redzone.get("modelScore")
        risk_score = None
        if isinstance(model_score, list) and model_score:
            first = model_score[0] if isinstance(model_score[0], dict) else None
            risk_score = first.get("riskScore") if first else None
        elif isinstance(model_score, dict):
            risk_score = model_score.get("riskScore")
        rows.append({"IBVStatusID": ibv_status_id, "redZoneV2_score": risk_score})
    except Exception as e:
        rows.append({"IBVStatusID": ibv_status_id, "redZoneV2_score": None})


df_redzone_scores = pd.DataFrame(rows).sort_values("IBVStatusID").reset_index(drop=True)

print(df_redzone_scores.head(10))
print(df_redzone_scores.shape)


   IBVStatusID  redZoneV2_score
0          757       145.000000
1         1258       162.000000
2         1382        78.979465
3         1810       298.000000
4         2334        94.987807
5         3158       224.000000
6         3509       216.000000
7         3513        61.982032
8         3586       154.000000
9         3854       146.000000
(11474, 2)


### Get the metrics dataframe

In [40]:
from sqlalchemy import create_engine 
import urllib
server = '192.168.1.15'
username = 'Junchen'
password = '9DFXjhhlR3k5'
database = 'LF_LMSMaster'

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server},1433;DATABASE={database};UID={username};PWD={password};"
    "Encrypt=yes;TrustServerCertificate=yes;"
)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


cnxn = engine.connect()
query1 = """ 
USE LF_LMSMASTER
DROP TABLE IF EXISTS #t1;
SELECT
    ibv.IBVStatusID,
    ibv.DateCreated AS IBVStatus_DateCreated,
    A.Application_ID, A.ApplicationSteps, D.DenialCode, D.DenialDescription, A.PortfolioID, A.CustomerID, A.CustomerSSN, A.ApplicationDate, A.Score, CustEmail,
    CASE WHEN ApplicationSteps LIKE '%S%' THEN 1 ELSE 0 END AS NewlyScored,
    CASE WHEN A.ApplicationStatus IN ('A','P') THEN 1 ELSE 0 END AS Accepted,
    CASE WHEN A.ApplicationStatus IN ('A','P') THEN A.LeadPurchasePrice ELSE 0 END AS LeadPurchasePrice,
    CASE WHEN L.LoanStatus NOT IN ('V','W','G','K') AND A.ApplicationStatus='J' THEN 1 ELSE 0 END AS PartialPreApproved,
    CASE WHEN L.LoanStatus NOT IN ('V','W','G','K') THEN 1 ELSE 0 END AS Originated,
    L.LoanID,
    DATEDIFF(YEAR, VW.DOB, A.ApplicationDate) AS Age,
    CASE WHEN VW.Frequency IN ('B','S') THEN 'B' ELSE VW.Frequency END AS Frequency,
    CASE WHEN L.LoanStatus NOT IN ('V','W','G','K') THEN L.OriginatedAmount ELSE NULL END AS OriginatedAmount,
    OriginationDate,
    LoanStatus,
    CASE WHEN ((L.RenewalLoanId <> '0') OR (A.LPCampaign = 'RENEW')) THEN 'RENEWAL'
         WHEN ApplicationSteps NOT LIKE '%R%' THEN 'NEW'
         ELSE 'REPEAT' END AS CustType,
    RenewalLoanId
INTO #t1
FROM Application A
LEFT JOIN Loans L
  ON A.Application_ID = L.ApplicationID AND A.PortFolioID = L.PortFolioID
LEFT JOIN LF_LMS_Logs..VW_ApplicationDump VW
  ON A.APPGUID = VW.APPGUID
LEFT JOIN LF_LMSMASTER..DenialCode AS D ON A.DenialCode = D.DenialCode

OUTER APPLY (
  SELECT TOP 1 s.IBVStatusID, s.DateCreated
  FROM [LF_BankData].[dbo].[IBVStatus] s
  WHERE s.AccountNumber = A.CustomerSSN
  ORDER BY ABS(DATEDIFF_BIG(SECOND, s.DateCreated, A.ApplicationDate)) ASC,
           s.DateCreated DESC, s.IBVStatusID DESC
) ibv

where ((CustEmail is null) or  (CustEmail not in ( 'josh@concordadvice.com', 'tiffany.speedyservicing@gmail.com','bobby@speedyservicing.com',
'and.kom@example.com', 'and.tor020@example.com', 'and.tor050@example.com', 'and.torrc4@example.com',
'test@dot818.com', 'test@example.com', 'test@gmail.com', 'test@loanscanada.ca', 'test@test.com', 'test2@test.com',
'testerqad@gmail.com', 'testik@test.com', 'testteam@dmaassociatescorp.com', 'tiffany.speedyservicing@gmail.com')))




UPDATE #t1 -- update the renewal loanid from 0 to correct previous loanIDs
SET #t1.RenewalLoanId = B.RenewalLoanId
from #t1 A inner join (select * from
(select A.LoanID, L.LoanID as RenewalLoanId, row_number() over (partition by A.LoanID order by datediff(day, L.OriginationDate, A.OriginationDate) desc) as RN from #t1 A
inner join Application A2 on A.CustomerID = A2.CustomerID and A2.ApplicationDate < A.ApplicationDate
inner join Loans L on A2.Application_ID = L.ApplicationID and A2.PortFolioID = L.PortFolioID and L.OriginationDate < A.OriginationDate and L.LoanStatus not in ('W','V','G','K')
where A.CustType = 'RENEWAL' and A.RenewalLoanId = '0') K where RN=1) B on A.LoanID = B.LoanID



-- scoring result for PlaidUDW_v1 and NegativeDBModelLP_v1
drop table if EXISTS #t2
select *
into #t2 from
(select A.*, B1.ExtResponse as B1ExtResponse, B2.ExtResponse as B2ExtResponse,
NDScore, IBVScore, NDBand,IBVBand,
row_number() over (partition by A.Application_ID, A.PortfolioID order by case when B2.ExtResponse is not null and B1.ExtResponse <> '' then 1 else 0 end desc,B.ApplicationDate desc) as RN
from #t1 A
left join Application B on A.CustomerID = B.CustomerID and A.ApplicationDate >= B.ApplicationDate AND B.ApplicationSteps like '%S%'
left join ScoringPythonResult B1 CROSS APPLY OPENJSON(B1.ExtResponse) WITH (IBVScore int '$.ModelScore', IBVBand int '$.IBVBand')  on B.Application_ID = B1.ApplicationID and B.PortFolioID = B1.PortfolioID and B1.iLabel= 'IBVBand'
left join ScoringPythonResult B2 CROSS APPLY OPENJSON(B2.ExtResponse) WITH (NDScore int '$.ModelScore', NDBand int '$.NDBand') on B.Application_ID = B2.ApplicationID and B.PortFolioID = B2.PortfolioID and B2.iLabel= 'NDBand' ) K where RN=1


ALTER TABLE #t2
DROP COLUMN RN;


-- Originated -- FPDFA rate:
-- go through the payments for First Installment First Attempt whose due date was before today and payment status not in V:Voided/P:Pending/T:3rd party
-- payment Mode: not in ('V' VIP,'P' promotion ,'T' rebate,'H' manual rebate,'G' charges)
DROP TABLE IF EXISTS #LF_LoanDefault
SELECT L.LoanID, L.Application_ID AS Application_ID, A.APPGUID, A.ApplicationDate, A.ApplicationSteps, L.PortFolioID, L.LoanStatus,
P.InstallmentNumber, P.PaymentStatus, P.PaymentType, P.PaymentMode, P.AttemptNo, P.TransactionDate, P.PaymentID,
I.InstallmentID, I.iPaymentMode, I.DueDate, I.Status, -- used to exclude pendings (code 684)
(CASE WHEN I.Status=684 THEN 1 ELSE 0 END) AS Pending
INTO #LF_LoanDefault
FROM #t2 L
LEFT JOIN LF_LMSMaster..Payment P ON P.LoanID = L.LoanID
LEFT JOIN LF_LMSMaster..Installments I ON I.InstallmentID = P.InstallmentID
LEFT JOIN LF_LMSMaster..Application A ON A.PortfolioID=L.PortfolioID AND A.Application_ID = L.Application_ID
WHERE I.InstallmentNumber = 1;

DROP TABLE IF EXISTS #LoanDefault_Flag
SELECT 
    L.*,
    -- FPDFA flag
    CASE 
        WHEN L.PaymentStatus = 'R'
             AND L.PaymentType IN ('I','S','A')
             AND L.PaymentMode IN ('A','B','D')
             AND L.DueDate <= CAST(GETDATE() AS date)
             AND NOT EXISTS (
                 SELECT 1
                 FROM #LF_LoanDefault ld
                 WHERE ld.InstallmentID = L.InstallmentID
                   AND ld.PaymentStatus = 'D'
                   AND ld.PaymentType NOT IN ('3','~','Q')
                   AND ld.PaymentMode IN ('A','D','B')
                   AND CONVERT(date, ld.TransactionDate) = CONVERT(date, L.DueDate)
             )
        THEN 1 ELSE 0 END AS is_FPDFA,
        CASE 
        WHEN L.LoanStatus NOT IN ('V','W','G','K')
             AND NOT (
                 L.iPaymentMode = 144 
                 AND L.Pending = 1
                 AND L.DueDate >= CAST(GETDATE() AS date)
             )
        THEN 1 ELSE 0 
    END AS is_loan_first_install
INTO #LoanDefault_Flag
FROM #LF_LoanDefault L;


DROP TABLE IF EXISTS #LoanDefault_Dedup;
WITH dedup AS (
    SELECT LoanID, Application_ID, APPGUID, ApplicationDate, ApplicationSteps, PortfolioID, LoanStatus,
           InstallmentNumber, PaymentStatus, PaymentType, PaymentMode, AttemptNo, TransactionDate,
           PaymentID, InstallmentID, iPaymentMode, DueDate, Status, Pending, is_FPDFA, is_loan_first_install,
           ROW_NUMBER() OVER (PARTITION BY Application_ID, PortfolioID ORDER BY is_FPDFA DESC) AS rn
    FROM #LoanDefault_Flag
)
SELECT LoanID, Application_ID, APPGUID, ApplicationDate, ApplicationSteps, PortfolioID, LoanStatus,
       InstallmentNumber, PaymentStatus, PaymentType, PaymentMode, AttemptNo, TransactionDate,
       PaymentID, InstallmentID, iPaymentMode, DueDate, Status, Pending, is_FPDFA, is_loan_first_install
INTO #LoanDefault_Dedup
FROM dedup
WHERE rn = 1;
-- dedup has 19151 unique loans -----

/* ----------
recreate #t3 using the new FPDFA logic 
-------------*/
drop table if EXISTS #t3 
select B.is_FPDFA AS FPDFA,  B.is_loan_first_install, B.PaymentStatus, A.* 
into #t3
from #t2 A 
left join #LoanDefault_Dedup B on A.LoanID = B.LoanID



DROP TABLE IF EXISTS #t4;
SELECT
    A.Application_ID,
    A.PortfolioID,
    A.LoanID,

    SUM(CASE WHEN P.PaymentStatus = 'D' THEN P.PaymentAmount ELSE 0 END) AS PaidOffPaymentAmount,
    SUM(CASE WHEN P.PaymentStatus = 'D' AND P.PaymentType IN ('X','~','3','W','B','K')
             THEN P.PaymentAmount ELSE 0 END) AS PaidOffCollAmount,
    SUM(CASE WHEN P.PaymentStatus = 'D' THEN P.FeeChargePaid ELSE 0 END) AS PaidOffFeeAmount,
    SUM(CASE WHEN P.PaymentStatus = 'D' THEN P.PaymentAmount - P.FeeChargePaid ELSE 0 END) AS PaidOffOtherThanFeeAmount,
    SUM(CASE WHEN P.PaymentStatus = 'R' THEN 1 ELSE 0 END) AS PmtReturn,
    COUNT(*) AS PmtCount
INTO #t4
FROM #t2 A
INNER JOIN LF_LMSMaster..Payment P
    ON A.LoanID = P.LoanID
   AND P.PaymentMode NOT IN ('V','P','T','H','G')
   AND P.PaymentType IN ('I','S','Z','A','X','Q','~','3','W','B','K')
   AND P.InstallmentNumber >= 1
   AND P.PaymentDate <= GETDATE()
   AND P.PaymentStatus IN ('D','R')
   AND P.PaymentAmount > 2.95
   AND P.PaymentAmount >= P.FeeChargePaid
GROUP BY
    A.Application_ID,
    A.PortfolioID,
    A.LoanID;



-- Big summarizing data table for Python processing
DROP TABLE IF EXISTS #t_app_summary;
SELECT A.*,B.FPDFA,B.is_loan_first_install, C.PaidOffPaymentAmount, C.PmtCount, C.PmtReturn,CASE
  WHEN A.OriginatedAmount IS NULL THEN 0                
  WHEN A.OriginatedAmount = 0 THEN 0.0                     
  ELSE COALESCE(C.PaidOffPaymentAmount, 0.0) 
       / CAST(A.OriginatedAmount AS float)
END AS Payin,
CASE
      WHEN A.OriginatedAmount IS NULL OR A.OriginatedAmount = 0 THEN 0
      WHEN COALESCE(C.PaidOffPaymentAmount, 0.0) >= A.OriginatedAmount * 1.05 THEN 1
      ELSE 0
    END AS IsGood
INTO #t_app_summary
from #t2 A  -- avg(OriginatedAmount+0.0) as AvgLoanAmount, sum(FPDFA) as FPDFA, sum(FPDFA+0.0)/count(*)*100 as FPDFARate
left join #t3 B on A.Application_ID = B.Application_ID and A.PortFolioID = B.PortFolioID
left join #t4 C on A.Application_ID = C.Application_ID and A.PortFolioID = C.PortFolioID
--where A.Originated = 1;



DROP TABLE IF EXISTS #ibv_to_apps;
SELECT
    rz.IBVStatusID,

       /* app identifiers */
    t1.Application_ID,
    t1.PortfolioID,
    t1.CustomerID,
    t1.CustomerSSN,
    t1.ApplicationDate,
    t1.ApplicationSteps,
    t1.DenialCode,
    t1.DenialDescription,
    t1.LoanID,

    /* model scores */
    t1.NDScore,
    t1.IBVScore,
    t1.NDBand,
    t1.IBVBand,

    /* performance */
    t1.Accepted,
    t1.FPDFA,
    t1.is_loan_first_install,
    t1.PaidOffPaymentAmount,
    t1.PmtCount,
    t1.PmtReturn,
    t1.Payin,
    t1.IsGood,
    t1.Originated,
    t1.OriginatedAmount,
    t1.OriginationDate,
    t1.LoanStatus,
    t1.CustType,
    t1.RenewalLoanId
INTO #ibv_to_apps
FROM (
    SELECT DISTINCT
        CAST(IBVToken AS BIGINT) AS IBVStatusID
    FROM BankuityPostOnboarding.dbo.SpeedyAnalysis
    WHERE ExperimentName = 'loonie_rerun_V3'
) rz
LEFT JOIN #t_app_summary t1
  ON t1.IBVStatusID = rz.IBVStatusID;
"""
query2 = """
SELECT *
FROM #ibv_to_apps

"""

with engine.begin() as cnxn:  # same connection + transaction
    cnxn.exec_driver_sql("SET NOCOUNT ON; USE LF_LMSMASTER;")
    cnxn.exec_driver_sql(query1)               # multi-statement OK here
    df_perf_orig = pd.read_sql(query2, cnxn)        # sees temp tables
# df_perf = pd.read_csv('perf_accept_orig_formodeling_dependentmeasure.csv')
df_perf_orig.rename(columns = {'Application_ID':'AppID'}, inplace=True)

print(df_perf_orig.shape)

(23405, 28)


In [32]:
df_perf_orig.columns

Index(['IBVStatusID', 'AppID', 'PortfolioID', 'CustomerID', 'CustomerSSN',
       'ApplicationDate', 'ApplicationSteps', 'DenialCode',
       'DenialDescription', 'LoanID', 'NDScore', 'IBVScore', 'NDBand',
       'IBVBand', 'Accepted', 'FPDFA', 'is_loan_first_install',
       'PaidOffPaymentAmount', 'PmtCount', 'PmtReturn', 'Payin', 'IsGood',
       'Originated', 'OriginatedAmount', 'OriginationDate', 'LoanStatus',
       'CustType', 'RenewalLoanId'],
      dtype='object')

In [33]:
# NA counts for core metrics
#df_perf_orig['is_loan_first_install'].value_counts()
df_perf_orig[['AppID','PortfolioID']].isna().sum()


AppID          1104
PortfolioID    1104
dtype: int64

In [34]:
# Inspect dtypes and sample value types for IBVStatusID keys
try:
    s = df_perf_orig["IBVStatusID"]
    sv = s.dropna().head(5)
    print("df_perf['IBVStatusID'] dtype:", s.dtype)
    print("df_perf samples:", list(sv))
    print("df_perf sample types:", [type(v).__name__ for v in sv])
except Exception as e:
    print("df_perf IBVStatusID check error:", e)

print()

try:
    s2 = df_redzone_scores["IBVStatusID"]
    sv2 = s2.dropna().head(5)
    print("df_redzone_scores['IBVStatusID'] dtype:", s2.dtype)
    print("df_redzone_scores samples:", list(sv2))
    print("df_redzone_scores sample types:", [type(v).__name__ for v in sv2])
except Exception as e:
    print("df_redzone_scores IBVStatusID check error:", e)


df_perf['IBVStatusID'] dtype: int64
df_perf samples: [80985, 52202, 80993, 64969, 81007]
df_perf sample types: ['int', 'int', 'int', 'int', 'int']

df_redzone_scores['IBVStatusID'] dtype: Int64
df_redzone_scores samples: [757, 1258, 1382, 1810, 2334]
df_redzone_scores sample types: ['int64', 'int64', 'int64', 'int64', 'int64']


#### normalize the join key -- IBVStatusID

In [35]:

df_perf_orig["IBVStatusID"] = df_perf_orig["IBVStatusID"].astype("Int64")
df_redzone_scores["IBVStatusID"] = pd.to_numeric(df_redzone_scores["IBVStatusID"], errors="coerce").astype("Int64")

### Join the RedZone Score to df_perf

In [36]:
# --- 0) align dtypes on join key ---
df_perf_orig["IBVStatusID"] = pd.to_numeric(df_perf_orig["IBVStatusID"], errors="coerce").astype("Int64")
df_redzone_scores["IBVStatusID"] = pd.to_numeric(df_redzone_scores["IBVStatusID"], errors="coerce").astype("Int64")

# --- 1) (optional) quick uniqueness checks ---
print("df_perf_orig rows:", len(df_perf_orig),
      "| unique IBVStatusID:", df_perf_orig["IBVStatusID"].nunique(),
      "| dup IBVStatusID rows:", df_perf_orig["IBVStatusID"].duplicated().sum())

print("df_redzone_scores rows:", len(df_redzone_scores),
      "| unique IBVStatusID:", df_redzone_scores["IBVStatusID"].nunique(),
      "| dup IBVStatusID rows:", df_redzone_scores["IBVStatusID"].duplicated().sum())

# --- 2) merge (base = df_perf_orig; append RedZone columns) ---
df_perf_merged = df_perf_orig.merge(
    df_redzone_scores[["IBVStatusID", "redZoneV2_score"]],
    on="IBVStatusID",
    how="left",
)

# --- 3) monitoring metrics ---
n = len(df_perf_merged)
n_matched = df_perf_merged["redZoneV2_score"].notna().sum()
n_missing = df_perf_merged["redZoneV2_score"].isna().sum()

print("\nJoin coverage (RedZoneV2_score not null):")
print("rows:", n)
print("matched:", n_matched)
print("missing:", n_missing)
print("match_pct:", round(100 * n_matched / n, 2), "%")

# (optional) how many IBVStatusIDs in df_perf_orig have no match in df_redzone_scores
no_match_ids = df_perf_merged.loc[df_perf_merged["redZoneV2_score"].isna(), "IBVStatusID"].dropna().unique()
print("unique IBVStatusIDs with no RedZone match:", len(no_match_ids))

df_perf_orig rows: 23405 | unique IBVStatusID: 11474 | dup IBVStatusID rows: 11931
df_redzone_scores rows: 11474 | unique IBVStatusID: 11474 | dup IBVStatusID rows: 0

Join coverage (RedZoneV2_score not null):
rows: 23405
matched: 23405
missing: 0
match_pct: 100.0 %
unique IBVStatusIDs with no RedZone match: 0


#### examine the merged data frame (Monitor Dashboard)

In [37]:

def monitor_score_coverage(df, id_col="IBVStatusID", rz_col="redZoneV2_score", ibv_col="IBVScore"):
    d = df.copy()

    # normalize types
    d[id_col] = pd.to_numeric(d[id_col], errors="coerce").astype("Int64")
    d[rz_col] = pd.to_numeric(d[rz_col], errors="coerce")
    d[ibv_col] = pd.to_numeric(d[ibv_col], errors="coerce")

    # row-level masks
    rz_ok = d[rz_col].notna()
    ibv_ok = d[ibv_col].notna()

    # ID-level coverage (any row for that ID has a score)
    ids = d[[id_col]].drop_duplicates()
    rz_ids = d.loc[rz_ok, [id_col]].drop_duplicates()
    ibv_ids = d.loc[ibv_ok, [id_col]].drop_duplicates()
    both_ids = d.loc[rz_ok & ibv_ok, [id_col]].drop_duplicates()

    n_ids = ids[id_col].nunique(dropna=True)

    print("=== Coverage summary ===")
    print("rows:", len(d))
    print("unique IBVStatusIDs:", n_ids)

    print("\nRow-level (may count multiple rows per ID):")
    print("rows with RedZone:", int(rz_ok.sum()))
    print("rows with IBVScore:", int(ibv_ok.sum()))
    print("rows with BOTH:", int((rz_ok & ibv_ok).sum()))
    print("rows with neither:", int((~rz_ok & ~ibv_ok).sum()))

    print("\nID-level (any row per IBVStatusID has the score):")
    print("IDs with RedZone:", rz_ids[id_col].nunique(), f"({rz_ids[id_col].nunique() / n_ids:.2%})")
    print("IDs with IBVScore:", ibv_ids[id_col].nunique(), f"({ibv_ids[id_col].nunique() / n_ids:.2%})")
    print("IDs with BOTH:", both_ids[id_col].nunique(), f"({both_ids[id_col].nunique() / n_ids:.2%})")

    # 4-way distribution at ID level
    rz_set = set(rz_ids[id_col].dropna().tolist())
    ibv_set = set(ibv_ids[id_col].dropna().tolist())
    all_set = set(ids[id_col].dropna().tolist())

    both = len(rz_set & ibv_set)
    rz_only = len(rz_set - ibv_set)
    ibv_only = len(ibv_set - rz_set)
    neither = len(all_set - (rz_set | ibv_set))

    print("\nID-level distribution:")
    print("both present:", both)
    print("RedZone only:", rz_only)
    print("IBVScore only:", ibv_only)
    print("neither:", neither)

# run it
monitor_score_coverage(df_perf_merged)

=== Coverage summary ===
rows: 23405
unique IBVStatusIDs: 11474

Row-level (may count multiple rows per ID):
rows with RedZone: 23405
rows with IBVScore: 19858
rows with BOTH: 19858
rows with neither: 0

ID-level (any row per IBVStatusID has the score):
IDs with RedZone: 11474 (100.00%)
IDs with IBVScore: 9406 (81.98%)
IDs with BOTH: 9406 (81.98%)

ID-level distribution:
both present: 9406
RedZone only: 2068
IBVScore only: 0
neither: 0


## Performance Table Coverage Review (RedZone Vs. IBV)

### Valid AppID + PortfolioID Scenario

In [28]:
# Peek examples: RedZone present but IBVScore missing (row + ID level)

import pandas as pd

# Peek examples: RedZone present but IBVScore missing
# Focus on cases where AppID+PortfolioID are present (ignore missing app keys).

required = ["IBVStatusID", "redZoneV2_score", "IBVScore"]
missing_cols = [c for c in required if c not in df_perf_merged.columns]
if missing_cols:
    raise ValueError(f"df_perf_merged missing columns: {missing_cols}")

d = df_perf_merged.copy()

# normalize a few types (keep the rest of the row intact)
for c in ["IBVStatusID", "AppID", "PortfolioID"]:
    if c in d.columns:
        d[c] = pd.to_numeric(d[c], errors="coerce").astype("Int64")

d["redZoneV2_score"] = pd.to_numeric(d["redZoneV2_score"], errors="coerce")
d["IBVScore"] = pd.to_numeric(d["IBVScore"], errors="coerce")

rz_ok = d["redZoneV2_score"].notna()
ibv_missing = d["IBVScore"].isna()
row_mask = rz_ok & ibv_missing

print("=== RedZone present, IBVScore missing (focused) ===")
print("rows (df_perf_merged):", len(d))
print("row-level: RedZone ok & IBV missing:", int(row_mask.sum()))

# Focus: AppID+PortfolioID present
if ("AppID" in d.columns) and ("PortfolioID" in d.columns):
    appkey_present = d["AppID"].notna() & d["PortfolioID"].notna()
    focused_mask = row_mask & appkey_present

    focused_ids = d.loc[focused_mask, "IBVStatusID"].dropna().drop_duplicates()
    print("row-level (AppID+PortfolioID present):", int(focused_mask.sum()))
    print("ID-level unique IBVStatusID (AppID+PortfolioID present):", int(focused_ids.shape[0]))
else:
    focused_mask = row_mask
    print("WARNING: AppID/PortfolioID not found; using row_mask without app-key filter.")

examples = (
    d.loc[focused_mask]
     .sort_values(["IBVStatusID"])
     .drop_duplicates("IBVStatusID")
     .head(15)
)

print("\nExamples (up to 15 unique IBVStatusIDs; full rows):")
display(examples)

if "CustType" in d.columns:
    tmp = d.loc[focused_mask, ["IBVStatusID", "CustType"]].drop_duplicates("IBVStatusID")
    print("\nCustType distribution among focused RedZone-only IDs (top 10):")
    display(tmp["CustType"].value_counts(dropna=False).head(10))


=== RedZone present, IBVScore missing (focused) ===
rows (df_perf_merged): 22017
row-level: RedZone ok & IBV missing: 3226
row-level (AppID+PortfolioID present): 1954
ID-level unique IBVStatusID (AppID+PortfolioID present): 1176

Examples (up to 15 unique IBVStatusIDs; full rows):


,IBVStatusID,AppID,PortfolioID,CustomerID,CustomerSSN,ApplicationDate,ApplicationSteps,DenialCode,DenialDescription,LoanID,...,PmtReturn,Payin,IsGood,Originated,OriginatedAmount,OriginationDate,LoanStatus,CustType,RenewalLoanId,redZoneV2_score
5684,1258,107753,1,58644.0,1570.0,2025-08-27 08:51:34.490,UU,43.0,Check Monthly Income,None,...,NaN,0.0,0.0,0.0,NaN,NaT,None,NEW,None,162.000000
5439,4152,71390,1,41618.0,5663.0,2025-05-08 10:41:16.180,UU,45.0,Home Phone Check,None,...,NaN,0.0,0.0,0.0,NaN,NaT,None,NEW,None,28.993583
17722,6346,108466,1,6183.0,8446.0,2025-08-29 03:10:58.217,UU,5.0,Last 45 Days,None,...,NaN,0.0,0.0,0.0,NaN,NaT,None,NEW,None,152.000000
3672,6379,139674,1,6207.0,8485.0,2025-12-17 10:31:53.730,UU,113.0,No Checking acc,None,...,NaN,0.0,0.0,0.0,NaN,NaT,None,NEW,None,162.000000
16837,6605,136157,1,6382.0,8780.0,2025-12-05 20:59:24.103,None,18.0,Day Not Allowed,None,...,NaN,0.0,0.0,0.0,NaN,NaT,None,REPEAT,None,321.000000
1613,8614,97847,1,54256.0,11247.0,2025-07-25 09:36:36.380,UU,43.0,Check Monthly Income,None,...,NaN,0.0,0.0,0.0,NaN,NaT,None,NEW,None,131.957005
3517,11784,142956,1,72320.0,14626.0,2025-12-26 12:08:39.017,UU,30.0,SIN Validation,None,...,NaN,0.0,0.0,0.0,NaN,NaT,None,NEW,None,126.000000
20522,15859,136857,1,15550.0,19684.0,2025-12-08 15:21:26.570,UU,113.0,No Checking acc,None,...,NaN,0.0,0.0,0.0,NaN,NaT,None,NEW,None,93.988021
10059,16042,137819,1,1884.0,2472.0,2025-12-11 09:25:06.627,UU,113.0,No Checking acc,None,...,NaN,0.0,0.0,0.0,NaN,NaT,None,NEW,None,166.000000
14078,16989,137141,1,16615.0,20959.0,2025-12-09 13:04:47.703,UUS,34.0,Failed Scoring,None,...,NaN,0.0,0.0,0.0,NaN,NaT,None,NEW,None,105.000000



CustType distribution among focused RedZone-only IDs (top 10):


CustType
NEW       988
REPEAT    188
Name: count, dtype: int64

In [38]:
# Save focused RedZone-present + IBVScore-missing apps to CSV

out_dir = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/misc_out_data"
os.makedirs(out_dir, exist_ok=True)

# Recompute the same focused filter used above
_d = df_perf_merged.copy()
_d["redZoneV2_score"] = pd.to_numeric(_d["redZoneV2_score"], errors="coerce")
_d["IBVScore"] = pd.to_numeric(_d["IBVScore"], errors="coerce")

if ("AppID" in _d.columns) and ("PortfolioID" in _d.columns):
    _d["AppID"] = pd.to_numeric(_d["AppID"], errors="coerce")
    _d["PortfolioID"] = pd.to_numeric(_d["PortfolioID"], errors="coerce")
    appkey_present = _d["AppID"].notna() & _d["PortfolioID"].notna()
else:
    appkey_present = True

mask = (
    _d["redZoneV2_score"].notna()
    & _d["IBVScore"].isna()
    & appkey_present
)

out_df = _d.loc[mask].copy()

out_path = os.path.join(out_dir, "redzone_present_ibv_missing_appkey_presentV1.csv")
out_df.to_csv(out_path, index=False)

print("Saved:", out_path)
print("rows:", len(out_df))
print("unique IBVStatusID:", out_df["IBVStatusID"].nunique() if "IBVStatusID" in out_df.columns else "(no IBVStatusID col)")
print("unique (AppID,PortfolioID):", out_df[["AppID","PortfolioID"]].drop_duplicates().shape[0] if ("AppID" in out_df.columns and "PortfolioID" in out_df.columns) else "(no app key cols)")


Saved: /Users/starsrain/nov2025_concord/loonie_bankuity_rerun/misc_out_data/redzone_present_ibv_missing_appkey_presentV1.csv
rows: 2443
unique IBVStatusID: 1434
unique (AppID,PortfolioID): 2443


### Null AppID + PortfolioID Case

In [39]:
# peek appId and portfolioId all null but with RedZone (NO IBVScore)


out_dir = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/misc_out_data"
os.makedirs(out_dir, exist_ok=True)

_d = df_perf_merged.copy()
_d["redZoneV2_score"] = pd.to_numeric(_d["redZoneV2_score"], errors="coerce")
_d["IBVScore"] = pd.to_numeric(_d["IBVScore"], errors="coerce")

if ("AppID" in _d.columns) and ("PortfolioID" in _d.columns):
    _d["AppID"] = pd.to_numeric(_d["AppID"], errors="coerce")
    _d["PortfolioID"] = pd.to_numeric(_d["PortfolioID"], errors="coerce")
    appkey_both_missing = _d["AppID"].isna() & _d["PortfolioID"].isna()
else:
    raise ValueError("df_perf_merged is missing AppID/PortfolioID columns")

mask_null_appkey = (
    _d["redZoneV2_score"].notna()
    & _d["IBVScore"].isna()
    & appkey_both_missing
)

null_appkey_df = _d.loc[mask_null_appkey].copy()

print("=== Null AppID+PortfolioID, RedZone present, IBVScore missing ===")
print("rows:", len(null_appkey_df))
print("unique IBVStatusID:", null_appkey_df["IBVStatusID"].nunique() if "IBVStatusID" in null_appkey_df.columns else "(no IBVStatusID col)")

# Pick 15 example rows, prefer unique IBVStatusID if present
if "IBVStatusID" in null_appkey_df.columns:
    examples = (
        null_appkey_df.sort_values(["IBVStatusID"])
                    .drop_duplicates("IBVStatusID")
                    .head(15)
    )
else:
    examples = null_appkey_df.head(15)

print("\nExamples (up to 15; full rows):")
display(examples)

out_path = os.path.join(out_dir, "examples_redzone_present_ibv_missing_appkey_null_V1.csv")
examples.to_csv(out_path, index=False)
print("\nSaved examples CSV:", out_path)
print("saved rows:", len(examples))


=== Null AppID+PortfolioID, RedZone present, IBVScore missing ===
rows: 1104
unique IBVStatusID: 1104

Examples (up to 15; full rows):


,IBVStatusID,AppID,PortfolioID,CustomerID,CustomerSSN,ApplicationDate,ApplicationSteps,DenialCode,DenialDescription,LoanID,...,PmtReturn,Payin,IsGood,Originated,OriginatedAmount,OriginationDate,LoanStatus,CustType,RenewalLoanId,redZoneV2_score
7411,757,NaN,NaN,NaN,NaN,NaT,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaT,None,None,None,145.000000
9713,3158,NaN,NaN,NaN,NaN,NaT,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaT,None,None,None,224.000000
1863,4911,NaN,NaN,NaN,NaN,NaT,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaT,None,None,None,132.000000
1913,6441,NaN,NaN,NaN,NaN,NaT,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaT,None,None,None,61.982032
15515,6591,NaN,NaN,NaN,NaN,NaT,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaT,None,None,None,160.000000
2942,9783,NaN,NaN,NaN,NaN,NaT,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaT,None,None,None,101.985241
2910,11043,NaN,NaN,NaN,NaN,NaT,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaT,None,None,None,115.976684
2896,11430,NaN,NaN,NaN,NaN,NaT,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaT,None,None,None,188.751444
1814,12778,NaN,NaN,NaN,NaN,NaT,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaT,None,None,None,80.000000
22624,13693,NaN,NaN,NaN,NaN,NaT,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaT,None,None,None,298.000000



Saved examples CSV: /Users/starsrain/nov2025_concord/loonie_bankuity_rerun/misc_out_data/examples_redzone_present_ibv_missing_appkey_null_V1.csv
saved rows: 15
